In [21]:
import pandas as pd
import torch.nn as nn
import torch
from sklearn.model_selection import train_test_split
from src.model.model import TBN
from src.model.utils import get_team_tensors

In [22]:
class LinearWithDropout(nn.Module):
    def __init__(self, in_dim, out_dim, dropout_p=0.5, bias=True):
        super(LinearWithDropout, self).__init__()
        self.linear = nn.Linear(in_dim, out, bias=bias)
        self.dropout = nn.Dropout(p=dropout_p)
    
    def forward(self, x):
        x = self.linear(x)
        x = self.dropout(x)
        return x
    
class PlayerEmbedding(nn.Module):
    def __init__(self, emb_dim):
        super(PlayerEmbedding, self).__init__()
        
        self.linear_with_dropout = LinearWithDropout(in_features=26, out_features=emb_dim-1, dropout_p=0.5, bias=True)
        
    def forward(self, x):
        x = self.linear_with_dropout(x)
        
        return x
        
    
class TBN(nn.Module):
    def __init__(self, emb_dim=16):
        super(TBN, self).__init__()
        
        # player embedding block
        self.player_embedding = PlayerEmbedding(emb_dim=emb_dim)
        
        # query, key, value networks for multihead attention
        self.q1 = nn.Linear(emb_dim, emb_dim)
        self.k1 = nn.Linear(emb_dim, emb_dim)
        self.v1 = nn.Linear(emb_dim, emb_dim)
        
        # multihead attention block
        self.multihead_attention1 = nn.MultiheadAttention(embed_dim=emb_dim, num_heads=2)
        self.post_attention_linear1 = nn.Linear(emb_dim, emb_dim)
        
        # query, key, value networks for multihead attention
        self.q2 = nn.Linear(emb_dim, emb_dim)
        self.k2 = nn.Linear(emb_dim, emb_dim)
        self.v2 = nn.Linear(emb_dim, emb_dim)
        # multihead attention block
        self.multihead_attention2 = nn.MultiheadAttention(embed_dim=emb_dim, num_heads=2)
        self.post_attention_linear2 = nn.Linear(emb_dim, emb_dim)

        # query, key, value networks for multihead attention
        self.q3 = nn.Linear(emb_dim, emb_dim)
        self.k3 = nn.Linear(emb_dim, emb_dim)
        self.v3 = nn.Linear(emb_dim, emb_dim)
        # multihead attention block
        self.multihead_attention3 = nn.MultiheadAttention(embed_dim=emb_dim, num_heads=2)
        self.post_attention_linear3 = nn.Linear(emb_dim, emb_dim)

        # incorporating team information
        self.team_linear = nn.Linear(21+emb_dim, emb_dim)
        
        # prediction head
        self.prediction = nn.Linear(emb_dim+emb_dim, 2)
        
        # activation function
        self.act = nn.ReLU()
        
    def forward(self, home_players, home_team, away_players, away_team):
        # initial embedding
        home_players_embedding = self.player_embedding(home_players) # (num players on home team) x (player embedding dimnesion)
        away_players_embedding = self.player_embedding(away_players) # (num players on away team) x (player embedding dimnesion)
        
        # add a flag to distinguish home team players from away team players
        home_players_embedding = torch.cat((home_players_embedding, torch.ones(home_players_embedding.size(0), 1)), dim=1) # (num players on home team) x (player embedding dimnesion + 1)
        away_players_embedding = torch.cat((away_players_embedding, -torch.ones(away_players_embedding.size(0), 1)), dim=1) # (num players on away team) x (player embedding dimnesion + 1)
        # nonlinearity
        home_players_embedding = self.act(home_players_embedding) # (num players on home team) x (player embedding dimnesion + 1)
        away_players_embedding = self.act(away_players_embedding) # (num players on away team) x (player embedding dimnesion + 1)
        
        # concatenate teams' player embeddings along player dimension
        combined_players_embedding = torch.cat((home_players_embedding, away_players_embedding), dim=0) # (num players on both teams) x (player embed dimension + 1)
        # add dummy batch dimension for now since we are just doing SDG
        combined_players_embedding.unsqueeze_(dim=1) # (num players on both teams) x (batch dim (== 1)) x (player embed dimension + 1)
        
        # MHA 1
        # calculate query, key, and values for multihead_attention
        c_q = self.q1(combined_players_embedding)  # (num players on both teams) x (batch dim (== 1)) x (player embed dimension + 1)
        c_k = self.k1(combined_players_embedding)  # (num players on both teams) x (batch dim (== 1)) x (player embed dimension + 1)
        c_v = self.v1(combined_players_embedding)  # (num players on both teams) x (batch dim (== 1)) x (player embed dimension + 1)
        # nonlinearities
        c_q = self.act(c_q)
        c_k = self.act(c_k)
        c_v = self.act(c_v)
        # update player embeddings with multihead_attention
        combined_players_embedding, _ = self.multihead_attention1(c_q, c_k, c_v)  # (num players on both teams) x (batch dim (== 1)) x (player embed dimension + 1)
        # nonlinearity
        combined_players_embedding = self.act(combined_players_embedding) # (num players on both teams) x (batch dim (== 1)) x (player embed dimension + 1)
        combined_players_embedding = self.post_attention_linear1(combined_players_embedding)
        combined_players_embedding = self.act(combined_players_embedding)
    
        # MHA 2
        # calculate query, key, and values for multihead_attention
        c_q = self.q2(combined_players_embedding)  # (num players on both teams) x (batch dim (== 1)) x (player embed dimension + 1)
        c_k = self.k2(combined_players_embedding)  # (num players on both teams) x (batch dim (== 1)) x (player embed dimension + 1)
        c_v = self.v2(combined_players_embedding)  # (num players on both teams) x (batch dim (== 1)) x (player embed dimension + 1)
        # nonlinearities
        c_q = self.act(c_q)
        c_k = self.act(c_k)
        c_v = self.act(c_v)
        # update player embeddings with multihead_attention
        combined_players_embedding, _ = self.multihead_attention2(c_q, c_k, c_v)  # (num players on both teams) x (batch dim (== 1)) x (player embed dimension + 1)
        # nonlinearity
        combined_players_embedding = self.act(combined_players_embedding) # (num players on both teams) x (batch dim (== 1)) x (player embed dimension + 1)
        combined_players_embedding = self.post_attention_linear2(combined_players_embedding)
        combined_players_embedding = self.act(combined_players_embedding)
            
        # MHA 3
        # calculate query, key, and values for multihead_attention
        c_q = self.q3(combined_players_embedding)  # (num players on both teams) x (batch dim (== 1)) x (player embed dimension + 1)
        c_k = self.k3(combined_players_embedding)  # (num players on both teams) x (batch dim (== 1)) x (player embed dimension + 1)
        c_v = self.v3(combined_players_embedding)  # (num players on both teams) x (batch dim (== 1)) x (player embed dimension + 1)
        # nonlinearities
        c_q = self.act(c_q)
        c_k = self.act(c_k)
        c_v = self.act(c_v)
        # update player embeddings with multihead_attention
        combined_players_embedding, _ = self.multihead_attention3(c_q, c_k, c_v)  # (num players on both teams) x (batch dim (== 1)) x (player embed dimension + 1)
        # nonlinearity
        combined_players_embedding = self.act(combined_players_embedding) # (num players on both teams) x (batch dim (== 1)) x (player embed dimension + 1)
        combined_players_embedding = self.act(combined_players_embedding) # (num players on both teams) x (batch dim (== 1)) x (player embed dimension + 1)
        combined_players_embedding = self.post_attention_linear3(combined_players_embedding)
        combined_players_embedding = self.act(combined_players_embedding)
    
        # split combined embedding back into team embeddings
        home_players_embedding = combined_players_embedding[:home_players.size(0)]  # (num players) x (batch dim (== 1)) x (embed dimension + 1)
        away_players_embedding = combined_players_embedding[home_players.size(0):]  # (num players) x (batch dim (== 1)) x (embed dimension + 1)
        
        # turn all player vectors into one team vector by summing along player dimension
        home_aggregate_player_embedding = torch.sum(home_players_embedding, dim=0) # (batch dim (== 1)) x (embed dim + 1)
        away_aggregate_player_embedding = torch.sum(away_players_embedding, dim=0) # (batch dim (== 1)) x (embed dim + 1)
        
        # remove dummy dimension
        home_aggregate_player_embedding.squeeze_(dim=0) # (embed dim + 1)
        away_aggregate_player_embedding.squeeze_(dim=0) # (embed dim + 1)
        
        # for some reasom getting weird shapes
        home_team = home_team.reshape(-1)
        away_team = away_team.reshape(-1)
        
        # concatenate team vector calculated from player information with team vector from team stats
        home_player_team_embedding = torch.cat((home_aggregate_player_embedding, home_team), dim=0) # (player embed dim + (embed dim + 1))
        away_player_team_embedding = torch.cat((away_aggregate_player_embedding, away_team), dim=0) # (player embed dim + (embed dim + 1))

        # put concatenated vectors through another linear layer
        home = self.team_linear(home_player_team_embedding) # (team post embed dim)
        away = self.team_linear(away_player_team_embedding) # (team post embed dim)
        
        # nonlinearities
        home = self.act(home)
        away = self.act(away)
        
        # concatenate outputs to turn both teams' vectors into a single vector for the prediction head
        pred_in = torch.cat((home, away), dim=0) # (2 x team_post_embed_dim)
        
        # use to predict final scores
        score_pred = self.prediction(pred_in) # (2)
        
        # return prediction
        return score_pred

In [23]:
game_df = pd.read_csv('data/game_23.csv')
game_train_df, game_test_df = train_test_split(game_df, test_size=0.2)

In [24]:
model = TBN()
model.train()
optim = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
loss_fn = torch.nn.HuberLoss()

num_epochs = 10

for epoch in range(num_epochs):
    for index, row in game_train_df.iterrows():
        home = row['HOME']
        away = row['AWAY']
        
        home_target = row['HOME_PTS']
        away_target = row['AWAY_PTS']
        targets = torch.tensor([home_target, away_target])

        home_players, home_team, away_players, away_team = get_team_tensors(home, away)
        
        preds = model(home_players, home_team, away_players, away_team)
        
        loss = loss_fn(targets, preds)
        
        optim.zero_grad()
        loss.backward()
        optim.step()
        
    num_correct_predictions = 0
    for index, row in game_test_df.iterrows():
        home = row['HOME']
        away = row['AWAY']
        
        home_target = row['HOME_PTS']
        away_target = row['AWAY_PTS']
        home_victory = home_target > away_target

        home_players, home_team, away_players, away_team = get_team_tensors(home, away)
        
        preds = model(home_players, home_team, away_players, away_team)
        predicted_home_victory = preds[0].item() > preds[1].item()
        
        correct_prediction = predicted_home_victory == home_victory
        if correct_prediction:
            num_correct_predictions+=1
        
    print(f'EPOCH {epoch + 1} OF {num_epochs}: {num_correct_predictions} OF {len(game_test_df)} GAMES CORRECTLY PREDICTED')
        

EPOCH 1 OF 10: 127 OF 247 GAMES CORRECTLY PREDICTED
EPOCH 2 OF 10: 124 OF 247 GAMES CORRECTLY PREDICTED
EPOCH 3 OF 10: 147 OF 247 GAMES CORRECTLY PREDICTED
EPOCH 4 OF 10: 139 OF 247 GAMES CORRECTLY PREDICTED
EPOCH 5 OF 10: 152 OF 247 GAMES CORRECTLY PREDICTED
EPOCH 6 OF 10: 158 OF 247 GAMES CORRECTLY PREDICTED
EPOCH 7 OF 10: 151 OF 247 GAMES CORRECTLY PREDICTED
EPOCH 8 OF 10: 145 OF 247 GAMES CORRECTLY PREDICTED
EPOCH 9 OF 10: 141 OF 247 GAMES CORRECTLY PREDICTED
EPOCH 10 OF 10: 141 OF 247 GAMES CORRECTLY PREDICTED


In [25]:
home_players, home_team, away_players, away_team = get_team_tensors('den', 'bos')
model(home_players, home_team, away_players, away_team)

tensor([0.9885, 1.0009], grad_fn=<ViewBackward0>)

In [26]:
home_players, home_team, away_players, away_team = get_team_tensors('bos', 'den')
model(home_players, home_team, away_players, away_team)

tensor([0.9922, 0.9740], grad_fn=<ViewBackward0>)